# Lambda notebook metalanguage overview
### Author: Kyle Rawlins

__Last updated__: Feb 2025

The core of the lambda notebook is a flexible and extensible typed logical metalanguage implemented in python -- the __Lambda Notebook Metalanguage (LNM)__.  The metalanguage is python-like, and can be interacted with either by cell magics, or by a python API.  Expressions of the metalanguage are parsed into python representations that can be interacted with programmatically.  There are three main aspects of the metalanguage to document: (i) the type system, (ii) the class hierarchy that supports this implementation, and (iii) the parser for writing metalanguage expressions directly. This notebook goes over each of these at an overview level; many aspects of the metalanguage have detailed documentation in separate notebooks.

At the technical level, the core unit of the implementation of LNM is the `TypedExpr` class.  Every expression is implemented as an instance of one of its subclasses.  A `TypedExpr` has a type together with an identity and set of behaviors for that type, and consists structurally of 0 or more argument slots. The arguments slots describe subexpressions, and themselves will always be subclasses `TypedExpr`. A `TypedExpr` that has 0 subexpressions is either a term or an empty data structure (e.g. an empty set). The behaviors provided by a class can include simplification and reduction patterns, type inference, variable binding in subexpressions, LaTeX rendering, and more.

# Interacting with the metalanguage

The metalanguage has two units: statements and expressions.  Expressions evaluate to metalanguage objects (of class _TypedExpr_ or a subclass), and are the main unit of the metalanguage.  For example, the following lines are each expressions.  The first one is a variable of type _e_, the second applies a variable to a constant property, and the third applies a constant of type _e_ to a lambda expression.

    x_e
    P_<e,t>(x_e)
    (L x_e : Cat(x))(Josie_e)

__Statements__ consist of assignments to terms or object language elements -- they update the lexicon, variable namespace, and python namespace with the value of an expression.  The thing to the right of `=` is a LNM expression.  Two things can be assigned values: variables names, and lexical items.  A lexical item is notated by surround the expression with `||` on each side.  Each of the following is a statement that assigns one of the above examples to a variable:

    x = x_e
    tmp = P_<e,t>(x_e)
    c = (L x_e : Cat(x))(Josie_e)
    
The following example illustrates a lexical item assignment:

    ||cat|| = L x_e : Cat(x)

There are two ways to process metalanguage: via ipython magics, and via python functions.  To process a statement, there is an ipython 'magic' named __lamb__.  This magic works in both cell and inline mode.  In cell mode, the first line of an ipython cell should be `%%lamb`, and any following lines will be treated as metalanguage statements.  The following cell interprets the above 4 examples of LNM statements.

In [ ]:
%%lamb
x = x_e
tmp = P_<e,t>(x_e)
c = (L x_e : Cat_<e,t>(x))(Josie_e)
||cat|| = L x_e : Cat_<e,t>(x)

Each statement interpreted by the __lamb__ magic has four effects: (i) the expression to the right of the equals sign is interpreted by the LNM parser and converted to a python object that is a subclass of `TypedExpr`; (ii) the resulting object is printed; (iii) if the assignment is to a lexical item, an item with that name is added to the lexicon, and (iv) a variable with the name of the assignment target is exported to the python namespace if possible.

  * _Technical note_: if the variable name is shadowed by something in the namespace, exporting fails and a warning is printed. A lexical entry defined this way can be accessed via `lang.get_system().lexicon`.
  
In inline mode, a single line of a cell is prefaced with `%lamb`; the line is interpreted as a statement in the same way described above, with one caveat: the result is only printed if the inline magic is the last line in the cell.

In [ ]:
print("This is normal python code here")
print(repr(tmp)) # this accesses the variable defined above via python code
%lamb ||dog|| = L x_e : Dog_<e,t>(x)
print("This is still python code")

In [ ]:
dog # let's print this just to make sure.  As a reminder, this cell is ordinary python code.

__Expressions__ can be input using statements in a `lamb` magic as above, evaluated using the `%te` line magic, or directly evaluated in python using the function `te` (automatically imported from `lamb.lang`).  

The `te` line magic evaluates a single expression.  This has the main effect, if used as the last line of a cell, of just printing the output.

  * As a side effect, this magic stores the result of each evaluation to the python variable `_llast`.


In [ ]:
%te L x_e : Dog(x)

In [ ]:
_llast

Like all IPython line magics, the `%te` magic can be mixed with python to a limited extent, in order to do variable assignment:

In [ ]:
d = %te L x_e : Dog_<e,t>(x)
d

The `te` python function behaves similarly: it parses an LNM expression provided as a string, and returns an appropriate `TypedExpr`.  The result can be manipulated just like any python object.

In [ ]:
d = te("L x_e : Dog_<e,t>(x)") # directly assign a python variable this LNM expression.  This bypasses the lexicon.
d

In [ ]:
d(te("Joe_e"))

In [ ]:
d(te("Joe_e")).reduce_all() # this performs beta reduction on any lambda expressions

# Types

The type system will be familiar to anyone who has used a typed lambda calculus or logical system such as Ty2, and this documentation assumes some basic familiarity with typed lambda calculi.  The standard definition of this sort of type hierarchy looks something like this, where rule 2 defines functional types:

 1. $e$ and $t$ are types.  (_atomic types_)
 2. If $\alpha$ and $\beta$ are types, so is $\langle\alpha,\beta\rangle$. (_non-atomic types_)
 3. Nothing else is a type.

The rule in 2 is sometimes characterized as giving a *type constructor*: the function type constructor given two types, produces a functional type notated as above. The type system here is extensible and designed for practical use, so we won't work with a strict definition, but the basic ideas hold.  A type system in the lambda notebook consists of a set of atomic types and non-atomic type constructors that are recursively constructed.  The library provides three atomic types by default:

 * $t$ (truth-values)
 * $e$ (entities)
 * $n$ (numbers)

The library also provides several non-atomic type constructors beyond just functions, including:

 * $\langle\alpha,\beta\rangle$: functional types, where the left is the input type and the right the output type.
 * $(\alpha_0,...\alpha_n)$: tuple types, for building tuples from atomic (or non-atomic) types.  In practice, what are often treated in logic as n-ary constants are treated here as functions from n-ary tuples to truth-values.
 * $\{\alpha\}$: set types, for sets of elements of type $\alpha$.
 * $[\alpha_0,...\alpha_n]$: a disjunctive type, which can match any of the listed elements.
 
The library also provides a mechanism for type polymorphism with type variables, what are written as $X$, $Y$, with any number of primes following or a number.

All types are interpreted and parsed relative to a type system, and in `meta` a current type system is defined; this can be accessed with `meta.get_type_system()`.  A convenience function `meta.tp` (imported into the notebook's namespace) calls the type system's parser; this has a corresponding line magic, `%tp`.  By default, the type system is `types.poly_system`.

In [ ]:
t = %tp <<e,t>,t>
t

In [ ]:
t.functional()

In [ ]:
t.left

In [ ]:
t.right

In [ ]:
t.right.functional()

In [ ]:
%tp <(e,e),t>

Atomic types correspond to sets of domain elements; see the notebook "Domain elements and MetaTerms" for detailed documentation on this. The type `t` consists of the set `{True, False}`. The type `n` corresponds to a set containing any python numeric value. This set is notionally non-finite (in that in principle, the set of python numerics is finite up to memory and precision limits, but that the domain set implementation acts as if it is non-finite). By default, new domains do not have custom domain sets like this, and are modeled using strings. For example, The set `e` is modeled via python strings that have a prefix `_c` followed by a number. In LaTeX, domain elements are rendered using sans-serif:

In [ ]:
%te (_c1, True, 10)

A few examples of type parse failures:

In [ ]:
%tp q
%tp <e,t
%tp <e,t>>

## Adding new types

Type systems have an api for adding basic types (and also writing recursive types), illustrated below. (There is also a convenience wrapper function for this that is a method on `lamb.lang` composition systems, `add_basic_type`.)

In [ ]:
meta.get_type_system().add_atomic(types.BasicType("s"))
display(meta.get_type_system())

By default the domain set for a new atomic type is modeled as a non-finite set of strings with a unique prefix similar to type `e` as described above, which will be inherited from the type name itself. (This can be manually specified by the `name` parameter to the `BasicType` constructor.) As a convenience, type `s` will automatically get the `w` prefix by default (similar for `v` and the prefix `e`, for eventive types):

In [ ]:
%te _w1

In [ ]:
types.reset() # use this to get back to the beginning

Not documented here: Adding a recursive type is possible but is a much more complicated matter, and typically requires modifying the metalanguage as well. Similarly, adding a different backing class for type domains is also possible, but complicated.

## Type variables

The default type system supports type variables; these are notated `X`, `Y`, `Z`, and can be suffixed with numbers or primes. See the notebook "Intro to type variables" for more on how these work and how to use them.  Within certain limits, a type variable stands in for arbitrary types.

In [ ]:
%tp <e,X>

In [ ]:
meta.get_type_system().unify(tp("<e,X>"), (tp("<e,t>")))

## Comparing types

The type system supports two forms of comparison: syntactic equality, and a semantic notion of equality called "unification" that is grounded in the theory of types. For *simple types*, these two are equivalent. For type variables and complex types constructed with type variables, they aren't. This document won't go into details (see the type variable documentation for more), but will outline the interface.

Syntactic equality uses the standard python equality operator:

In [ ]:
tp("<e,t>") == tp("<e,t>"), tp("<e,t>") == tp("e"), tp("X") == tp("Y")

Unification is more complex.  Given two types, a `unify` check asks: is there a common type (termed a *principal type*) that is compatible with the two? You'll notice that above the types `X` and `Y` compare as not equal. However, there is a principal type that is compatible with both (in fact, more than one): for example, `X`. A type system comes with a `unify` function that does this check, giving you back a principal type. Here are a few examples:

In [ ]:
meta.get_type_system().unify(tp("X"), tp("Y"))

In [ ]:
meta.get_type_system().unify(tp("X"), tp("<e,t>"))

In [ ]:
meta.get_type_system().unify(tp("<X,t>"), tp("<<e,Y>,t>"))

In [ ]:
meta.get_type_system().unify(tp("<X,t>"), tp("<e,Y>"))

In general, to compare two types for purposes of semantic analysis, you should use unification, not syntactic equality.

# Metalanguage overview

There are four key kinds of `TypedExpr`s to worry about: terms (variables and constants), lambda expressions, operators, and binding operators.  (Lambda expressions are a special case of the latter, but are important enough to separate out.)

In [ ]:
meta.suppress_constant_predicate_type = False # normally, predicate types are hidden for prettier output.  But it is helpful here to be able to see them

### Terms

A regular term consists simply of a name and optionally a type.  Capitalized names are constants, and lower-case names are variables. A metaterm is a term that explicitly `refers` to a type domain element, such as a number or other domain element. See the notebook on domain elements and MetaTerms for detailed documentation on this concept. The following example illustrates an expression with three terms: a Predicate at type `<(e,n>,t>`, a variable at type `e`, and a numeric metaterm.

In [ ]:
%te P_<(e,n),t>(x_e, 1)

By default, if no type is specified, the parser will guess according to some simple heuristics: variables are of type `e` and constant functions are adjusted to be predicates of their arguments (with a base default of type `t`). (This is quite different than specifying a variable type.) The following example illustrates a predicate-argument combination with these defaults.

In [ ]:
%te P(x)

Types can be explicitly specified by putting a `_` after a term. The following is equivalent to the prior example:

In [ ]:
%te P_<e,t>(x_e)

When commas are used in the argument to a function, the argument is treated as an n-ary tuple.  This is the standard way to construct n-ary 'predicates' in LNM.

In [ ]:
e = %te P(x,y)
e

In [ ]:
e[0].type # show the predicate's type in the above example using the python API

There is an API for constructing expressions in python, without using LNM syntax.  The following cells provide some examples (but not comprehensive documentation).

To construct terms programmatically, use the term factory function instead of instantiating classes directly, which can be shortened to `meta.term`.  It is also often convenient to use the type parser here, though the factory function is fairly flexible.

In [ ]:
e3 = meta.term("P", tp("<(e,e),t>"))
e3

In [ ]:
meta.term("P_<(e,e),t>")

Many python operators are overloaded to properly handle TypedExprs, most importantly function calling.

In [ ]:
meta.term("P_<(e,e),t>")(meta.term("x_e"), meta.term("y_e"))

In [ ]:
meta.term("P_<e,<e,t>>")(meta.term("x_e"))(meta.term("y_e"))

### Lambda expressions

See also the documentation on *Metalanguage functions*.

Lambda expressions (class: `LFun`) consist of a typed variable (i.e. lower-case TypedTerm) and a body, which is a TypedExpr.  Instances of that variable name in the body (and not under the scope of some lower binder) are interpreted as bound by that lambda operator, and the types must much (or a TypeMismatch exception will result).  The type of the whole is the type of a function from the variable's type to the body's type.  They are notated as `L var_type : body`.  The parser will except several alternatives for `L`, including the more pythonic `lambda`.

In [ ]:
e3 = %te L y_e : P(y)
display(e3, e3.type)

Function-argument expressions can be easily written in the expected way using parenthesis:  (parenthesis around the function are obligatory)

In [ ]:
%te (L y_e : P(y))(x_e)

Note that beta-reduction does not happen automatically (although type-checking does).  There are a number of ways to force beta-reduction.  The simplest for displaying a reduction is to add `reduce` preceded and followed by a space to a `%te` magic.  The others are via python calls, and are explained below.

  * technical note: if you are using a variable named `reduce`, just ensure there is no space following it.

In [ ]:
%te reduce (L y_e : P(y))(x_e)

As with terms, the python function call notation works correctly.  'Correct' in this case, though, means that it will construct a composite TypedExpr with the lambda expression as the operator, and the argument as the operand.

In [ ]:
e3(te("x_e"))

To actually apply the argument to the function (and force beta reduction), there are two ways.  You can construct the expression using `apply`, or call the `reduce_all()` function on any `TypedExpr`.

In [ ]:
e3.apply(te("x_e"))

In [ ]:
e4 = e3(te("x_e")).reduce_all()
e4

The `e4` object here has a derivational history that we can inspect:

In [ ]:
e4.derivation

A somewhat more complicated example:

In [ ]:
e5 = %te L y_e : L x_e : P(x,y)
e5

In [ ]:
e6 = e5(te("A_e"))(te("B_e"))
e6

In [ ]:
e6.reduce_all().derivation

### Basic operator expressions

Logical operators (and numeric operators) are supported using python syntax for bitwise operations.  This should be mostly straightforward for anyone familiar with classical logic.  As you might expect, types are enforced.  The parsed syntax is (with a few exceptions) the same as the python syntax.

 * __numeric operators__: The operators `+` (unary and binary), `-` (unary and binary), `*`, `/`, and `**` all work as expected for type `n`, as well as `<`, `<=`, `>`, and `>=`.
 * __equality__: The metalanguage provides a semantic equivalence operator that can be written as `<=>`, `==`, or `%` in metalanguage code. At type `t`, this will specialize to a standard biconditional operator, but a generic version is present at any type. Of course, simplifying expressions of equivalence is highly non-trivial in the general case (though many special cases can be handled). Note that the `==` operator cannot be used on python objects, where it is implemented in a non-metalanguage use; the `%` operator can be used, as well as the member function `equivalent_to`. In metalanguage expressions, the non-equivalence operator can be used for boolean objects only, instantiated by `!=`, `=/=`, and `^`. Only the latter is available in python code.
 * __logical operators__: similarly, python's logical operators (`and`, `or`, `not`) do not work on metalanguage objects, and also cannot be used in metalanguage expressions. Rather, the operators `&` (and), `|` (or) and `~` (not) should be used. (These are python bitwise operators, and this setup is due to constraints in the python data model. This use of bitwise operators may also be familiar from packages like `pandas`, `sympy`, etc.) The material implication operator uses `>>`.  In the metalanguage (but not python), `==>` will also work.
    - A ternary conditional operator, named `Case`, can be used with a predicate-argument syntax. This operator takes a boolean condition, with an if and an else argument that must have matching types. Rather than a typical programming language `if...else...` construct, it is better thought of in terms of standard mathematical definition by cases, and is rendered like this in LaTeX.
 * __set operators__: See the set documentation for more on set operators.

__Operator precedence__: the operators currently all use [python precedence](https://docs.python.org/3/reference/expressions.html#operator-precedence), even in LNM. This will mostly work as you would expect, but there are occasional caveats. For example, because equality uses `%`, it takes corresponding precedence over most other operators. It is always safe to use parentheses to disambiguate.

Here are some examples of numeric operators:

In [ ]:
%te 2 + 2

In [ ]:
%te simplify 2 + 2
_llast.derivation

In [ ]:
%te simplify (L x_n :2 * x_n + 6)(5)
_llast.derivation

Here are some examples of boolean operators:

In [ ]:
%te p_t & (q_t | r_t)

In [ ]:
%te p_t >> ~q_t

In [ ]:
%te simplify q_t & (p_t & ~p_t)
_llast.derivation

In [ ]:
%te simplify True | (p_t & ~p_t)
_llast.derivation

In [ ]:
%te Case(p_t & ~p_t, 1, 2)

In [ ]:
%te simplify Case(p_t & ~p_t, 1, 2)
_llast.derivation

Function definition by cases:

In [ ]:
%te L x_n : Case(x == 5, 1, Case(x == 6, 2, 3))

In [ ]:
%te simplify (L x_n : Case(x == 5, 1, Case(x == 6, 2, 3)))(7)
_llast.derivation

Many operators are overloaded to work in python as well; the following instances of `&`, `|`, `>>`, and `~` are python.

In [ ]:
te("p_t") & (te("q_t") | te("r_t"))

In [ ]:
te("~p_t") >> ~te("q_t")

### Binding expressions

A number of operators use a similar syntax to lambda expressions, though with different type structures.  Existential and universal quantification are expressions of type $t$, that bind a variable of some type and whose body is type $t$.  Sets can also be defined using a similar notation, but their overall type is the type of a set whose elements are the variable's type.

In [ ]:
%te Exists x_e : P(x)

In [ ]:
%te Forall x_e : P(x)

In [ ]:
%te Set f_<e,t> : Exists x_e : f(x)

## Data structures

Tuples, sets, and a dict-like data structure are all supported.

* Sets are documented in a separate set-specific notebook.
* Map-based functions are documented in the function documentation notebook.

In [ ]:
%te (1, _c1, x_<e,t>) # 3-tuple of a mixed type

In [ ]:
%te {_c1, _c2, x_e} # set of type `{e}`

In [ ]:
%te {1:2, 2:x_n} # map function of type <n,n>

# Appendix: metalanguage class hierarchy

`TypedExpr`: base class for all typed expressions.

 * `SyncatOpExpr`: superclass for many operators, including logical operators, numerics, and set operators. Subclasses:
    - `BinaryAndExpr` (python / LNM: '`&`')
    - `BinaryOrExpr` (python / LNM: '`|`')
    - `BinaryArrowExpr` (python / LNM: '`>>`'.  __Warning__: this takes very high precedence, parens are recommended.  LNM only: '`==>`')
    - `BinaryBiarrowExpr` (LNM: '`<=>`' at type t)
    - `BinaryNeqExpr` (python/LNM: '`^`' or '`%`'  LNM only: '`=/=`')
    - `BinaryGenericEqExpr` (LNM: '`<=>`' at types other than t. __Warning__: '`==`' does not work on python objects.)
    - numeric expressions: `BinaryLExpr`, `BinaryLeqExpr`, `BinaryGeqExpr`, `BinaryGExpr`, `BinaryPlusExpr`, `BinaryMinusExpr`, `BinaryDivExpr`, `BinaryTimesExpr`, `BinaryExpExpr`
    - `UnaryNegExpr`: negation. (Python/LNM: '`~`')
    - `UnaryNegativeExpr`: numeric unary negative operator.  (Python/LNM: '`-`')
    - `SetContains` (python/LNM: '`<<`'.  __Warning__: precedence is tricky, parens are recommended.)
    - `BinarySetOp`. Superclass for many set operators: `SetUnion`, `SetIntersection`, `SetDifference`, `SetEquivalence`, `SetSubset`, `SetPropersubset`. (`SetSupset` and `SetPropersupset` are direct subclasses of `SyncatOpExpr`.) See the set-specific documentation notebook.
 * `BindingOp`: class for operators that bind a variable (notated `v` below) in their scope.  For many of these, there are shorthand expressions not documented here, and the unicode will also work.
    - `ConditionSet`: set defined in terms of a condition on some variable.  (LNM: '`Set v : `'
    - `Forall`: $\forall$ operator. (LNM: '`Forall v :`')
    - `Exists`: $\exists$ operator. (LNM: '`Exists v :`')
    - `ExistsExact`: $\exists !$ operator. (LNM: '`ExistsExact v :`')
    - `Iota`: $\iota$ operator (e.g. for definite descriptions).  (LNM: '`Iota v :`')
    - `LFun`: $\lambda$ expression, describing a function. (LNM: '`L v :`')
 * `Tuple`: a typed tuple, notated with python tuple syntax.  (Elements can be of arbitrary type, and needn't be uniform.)
 * `ListedSet`: set defined by listing elements, notated with python set syntax. Elements must have uniform type.
 * `MapFun`: a dict-like function object, notated with python dict syntax; see the function documentation notebook.
 * `ChainFun`: a composite function constructed from other functions, similar to python's `ChainMap`. LNM operator: `+`.
 * `Partial`: a class for instantiating partially defined metalanguage objects, e.g. partial LFuns.
    - The classes `Body` and `Condition` (direct subclasses of `TypedExpr`) correspond to operators that extract the body and condition of a partially defined expression.
 * `FunDom`, `FunCodom` are operators that get function domains/codomains; see the function documentation notebook.
 * `TypedTerm`: represents a typed term expression at arbitrary types (variable or constant). Terms that appear with this class are inherently "unvalued" in that even constant terms are not associated with a domain element value without a model providing it.
    - `MetaTerm`: class for referring directly to elements of type domains, such as numbers. `MetaTerm`s are inherently valued.